In [1]:
import csv
import openpyxl

In [13]:
#엑셀 파일 임시 생성
wb = openpyxl.Workbook()
#sheet활성화
sheet = wb.active
#sheet이름 작성
sheet.title = '병원 검색 결과'
#1행작성(컬럼명)
sheet.append(['시군구','의료기관명','전화번호'])

location = ''
hospital= ''
phone = ''   
#데이터가 있는지 확인하기 위해 리스트 선언
hos_list = []

while True:
    city = input('시군구를 입력하세요.: ')
    hos_list.clear() #리스트 삭제
        
    if city == '0':  # 0입력하면 종료
        print('종료합니다.')
        break;   
            
    if city != '0':
        print(city)
        #csv파일 가져오기
        f = open('./covid19hospitals.csv','r')
        reader = csv.DictReader(f)
        for row in reader:
            if city in row['시군구']:
                location = row['시군구']
                hospital = row['의료기관명']
                phone = row['전화번호']
                print(location,hospital,phone)
                #리스트에 담기
                hos_list.append([location,hospital,phone])
                #hospital_list.xlsx엑셀에 데이터 추가
                sheet.append([location,hospital,phone])
        #리스트가 비어있다면   
        if not hos_list: 
            print('데이터가 존재하지 않습니다.')
                
        
#print(hosList)
#csv 종료
f.close()
#엑셀 저장
wb.save('hospital_list.xlsx')
#엑셀에 넣은 데이터 보여주기
for row in sheet.iter_rows(min_row=1, max_col=3, values_only=True):
    print(row)

시군구를 입력하세요.: 동해
동해
강원도 동해시 근로복지공단동해병원 033)530-3100
강원도 동해시 동해동인병원 033)532-3006
시군구를 입력하세요.: 서울
서울
서울시 강남구 강남베드로병원 02-1544-7522
서울시 강동구 강동성모요양병원 02)488-0020
서울시 강북구 신일병원 02)903-5121
서울시 강서구 강서연세병원 02)2658-5114
서울시 강서구 강서힘찬병원 1899-2228
서울시 관악구 사랑의병원 02)880-0114
서울시 광진구 김종웅내과의원 02)455-4038
서울시 구로구 권내과의원 02)2685-6612
서울시 금천구 연세조내과 02)803-2134
서울시 금천구 서울으뜸정형외과의원 02)807-0111
서울시 노원구 밝은미래안과 02)939-3075
서울시 노원구 삼성드림이비인후과 02)935-1365
서울시 노원구 새서울병원 02)930-5858
서울시 노원구 어비뇨기과 02)930-0131
서울시 도봉구 도봉병원 02)3492-3250
서울시 동대문구 서울나은병원 1544-6003
서울시 동대문구 현대중앙의원 02)2244-9600
서울시 동작구 의료법인성석의료재단 동작경희병원 02)822-8112
서울시 마포구 박상수내과의원 02)332-5460
서울시 마포구 예담정형외과의원 02-335-2500
서울시 마포구 서울본내과의원 02)3143-2220
서울시 서대문구 한양그린내과의원 02)379-3377
서울시 서초구 김일중내과 02)3473-1356
서울시 성동구 9988병원 02)2297-9988
서울시 성북구 서울척병원 1599-0033
서울시 성북구 의료법인 유라의료재단 온누리요양병원 02)919-2700
서울시 송파구 서울수내과의원 02)414-9919
서울시 양천구 메디힐병원 02)2604-7551
서울시 영등포구 jc빛소망 안과 02)785-1068
서울시 영등포구 아이비이비인후과 02)784-7533
서울시 영등포구 윤문수비뇨기과 02)2069-0221
서울시 영등포구 의료법인 영등포병원 

In [14]:
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib

# SMTP 접속을 위한 서버, 계정 설정
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

# 보내는 메일 계정
SMTP_USER = "rabbitn402@gmail.com"
SMTP_PASSWORD = "teatree1234"

# 이메일 유효성 검사 함수
def is_valid(addr):
    import re
    if re.match('(^[a-zA-Z-0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)', addr):
        return True
    else:
        return False

# 이메일 보내기 함수
def send_mail(addr, subj_layout, cont_layout, attachment=None):
    if not is_valid(addr):
        print("Wrong email: " + addr)
        return
    
    # 텍스트 파일
    msg = MIMEMultipart("alternative")
    # 첨부파일이 있는 경우 mixed로 multipart 생성
    if attachment:
        msg = MIMEMultipart('mixed')

    msg["From"] = SMTP_USER
    msg["To"] = addr
    msg["Subject"] = subj_layout

    contents = cont_layout
    text = MIMEText(_text = contents, _charset = "utf-8")
    msg.attach(text)

    # 첨부파일이 있으면
    if attachment:
        from email.mime.base import MIMEBase
        from email import encoders

        file_data = MIMEBase("application", "octect-stream")
        file_data.set_payload(open(attachment, "rb").read())
        encoders.encode_base64(file_data)

        import os
        filename = os.path.basename(attachment)
        file_data.add_header("Content-Disposition", 'attachment', filename=('UTF-8', '', filename))
        msg.attach(file_data)

    # smtp로 접속할 서버 정보를 가진 클래스변수 생성
    smtp = smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT)

    # 해당 서버로 로그인
    smtp.login(SMTP_USER, SMTP_PASSWORD)

    # 메일 발송
    smtp.sendmail(SMTP_USER, addr, msg.as_string())

    # 닫기
    smtp.close()

In [15]:
from openpyxl import load_workbook

wb = load_workbook('email_list.xlsx')
ws = wb.active

for row in ws.iter_rows():
    addr = row[0].value
    subj_layout = row[1].value
    cont_layout = row[2].value
    attachment = row[3].value

    send_mail(addr, subj_layout, cont_layout, attachment)